In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate # show examples instead of prompt
from langchain.prompts.example_selector import LengthBasedExampleSelector # select examples based on length
from langchain.prompts.example_selector.base import BaseExampleSelector

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [15]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    def add_example(self, example):
        self.examples.append(example)
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
example_selector = LengthBasedExampleSelector(
    example_prompt=example_prompt,
    examples=examples,
    max_length=100,
    min_length=50,
)
example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    # examples
    example_prompt=example_prompt, # format
    example_selector=example_selector, # examples=examples,
    # user question
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
print(prompt.format(country="Germany"))

chain = prompt | llm
chain.invoke({"country": "Turkey"})

Human: What do you know about Italy?
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Germany?
AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira
        Famous landmarks: Hagia Sophia, Blue Mosque, Cappadocia

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira\n        Famous landmarks: Hagia Sophia, Blue Mosque, Cappadocia')